In [ ]:
%cd "Paciente 17"

/content/Paciente 17


In [ ]:
import pandas as pd

test = pd.read_csv('Teste.csv')
train = pd.read_csv('Treino.csv')

label_column = test.columns[-1]

Y_test = test[label_column]
Y_train = train[label_column]

X_test = test.drop(columns=label_column)
X_train = train.drop(columns=label_column)

In [ ]:
from sklearn.linear_model import LinearRegression


model = LinearRegression().fit(X_train,Y_train)
predictions = model.predict(X_test)

In [ ]:
import plotly.express as px


px.histogram(predictions, x=0, histnorm='percent')

In [ ]:
pred_label = pd.DataFrame({'label': Y_test, 'regression': predictions}).reset_index()
pred_label['label'] = pred_label['label'].map({1: 'Normal', -1: 'Extra Sistólico'})
pred_label['prediction'] = pred_label['regression'].apply(lambda num: num > 0).map({True: 'Normal', False: 'Extra Sistólico'})
pred_label['evaluation'] = (pred_label['label'] == pred_label['prediction']).map({True: 'Certo', False: 'Errado'})
pred_label

,index,label,regression,prediction,evaluation
0,0,Normal,1.360506,Normal,Certo
1,1,Normal,0.972819,Normal,Certo
2,2,Normal,1.045013,Normal,Certo
3,3,Normal,0.834774,Normal,Certo
4,4,Normal,0.883253,Normal,Certo
...,...,...,...,...,...
120,120,Extra Sistólico,0.472009,Normal,Errado
121,121,Extra Sistólico,-0.601273,Extra Sistólico,Certo
122,122,Extra Sistólico,1.760208,Normal,Errado
123,123,Extra Sistólico,-1.722000,Extra Sistólico,Certo


In [ ]:
import plotly.graph_objects as go

fig = px.scatter(
    pred_label,
    x='index',
    y='regression',
    color='label',
    symbol='evaluation',
    color_discrete_map={'Normal': 'darkolivegreen', 'Extra Sistólico': 'red'},
    symbol_map={'Certo': 'circle', 'Errado': 'x'},
)
fig.update_traces(marker={'size': 13})
fig.add_shape(type="line", line_dash="dash", x0=0, x1=pred_label.shape[0], y0=0, y1=0, line_color='darkblue')

In [ ]:
from sklearn.metrics import confusion_matrix
import plotly.figure_factory as ff


labels = ['Normal', 'Extra Sistólico']
conf_matrix = confusion_matrix(pred_label['label'], pred_label['prediction'], labels = ['Normal', 'Extra Sistólico'])

fig=ff.create_annotated_heatmap(conf_matrix, x=labels, y=labels, showscale=True, colorscale="tempo")
fig.update_layout(width=400, height=400)

fig

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(pred_label['label'], pred_label['prediction']))

                 precision    recall  f1-score   support

Extra Sistólico       0.76      0.73      0.75        26
         Normal       0.93      0.94      0.93        99

       accuracy                           0.90       125
      macro avg       0.84      0.84      0.84       125
   weighted avg       0.89      0.90      0.90       125



In [ ]:
from sklearn.metrics import classification_report
import seaborn as sns


clf_report = classification_report(pred_label['label'], pred_label['prediction'], output_dict=True)
col_order = ['Normal', 'Extra Sistólico', 'macro avg', 'weighted avg', 'accuracy']
clf_report = pd.DataFrame(clf_report)[col_order].drop(columns=['accuracy']).iloc[:-1, :].T

sns.heatmap(clf_report, annot=True, cmap="YlGnBu")

In [ ]:
import plotly.express as px

result = pd.concat([X_test, pred_label[['label', 'prediction']]], axis=1)
transform_data = pd.melt(result.reset_index(), id_vars=['label', 'index', 'prediction'])
transform_data =  transform_data.rename(columns={'index': 'amostra', 'label': 'batimento', 'value': 'amplitude', 'prediction': 'predito'}).reset_index()
transform_data['Tempo [(*1/360)]s'] = transform_data['index']/360


labels = {'amplitude': 'amplitude mv[100]'}
fig = px.line(
    transform_data,
    x='Tempo [(*1/360)]s',
    y='amplitude',
    color='amostra',
    facet_row='batimento',
    facet_col='predito',
    color_discrete_sequence=px.colors.qualitative.Prism, labels=labels,  width=480, height=450)
fig.update_layout(showlegend=False)
fig